In [41]:
import os
import json
from typing import List
from tqdm import tqdm
import torch
from torch import nn

from langchain.docstore.document import Document as LangchainDocument
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from transformers import AutoTokenizer, AutoModel
from pydantic import PrivateAttr

EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"


class MultiGPUHuggingFaceEmbeddings(HuggingFaceEmbeddings):
    _tokenizer: AutoTokenizer = PrivateAttr()
    _model: nn.Module = PrivateAttr()
    _device: str = PrivateAttr()
    _half: bool = PrivateAttr()
    _normalize_embeddings: bool = PrivateAttr(default=False)

    def __init__(self, model_name: str = EMBEDDING_MODEL_NAME, device: str = None, half: bool = True, **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        
        encode_kwargs = kwargs.get("encode_kwargs", {})
        self._normalize_embeddings = encode_kwargs.get("normalize_embeddings", True)  # включаем нормализацию

        self._tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._model = AutoModel.from_pretrained(model_name)
        self._model.eval()

        # Проверка на количество GPU
        if torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs for inference...")
            self._model = nn.DataParallel(self._model)
            self._device = 'cuda'
        else:
            self._device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")

        self._model.to(self._device)

        if half and 'cuda' in self._device:
            self._model.half()

        self._half = half

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # Используем mean pooling для получения sentence embeddings
        batch_size = 256
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            inputs = self._tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
            inputs = {k: v.to(self._device) for k,v in inputs.items()}

            with torch.no_grad():
                outputs = self._model(**inputs)
                last_hidden_state = outputs.last_hidden_state
                attention_mask = inputs['attention_mask']

                # Mean Pooling
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
                sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
                sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
                cls_emb = (sum_embeddings / sum_mask).float().cpu().numpy()

                if self._normalize_embeddings:
                    norm = (cls_emb**2).sum(axis=1, keepdims=True)**0.5
                    cls_emb = cls_emb / norm
                embeddings.extend(cls_emb.tolist())
        return embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]


In [54]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = MultiGPUHuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        half=True,
        model_kwargs={"device": "mps"},
        encode_kwargs={"normalize_embeddings": True},
        multi_process=False
    )

# Загрузка сохранённой базы данных из директории, где лежат index.faiss и index.pkl
vectorstore = FAISS.load_local("/Users/aleksandrlazarev/Documents/TaBERT/fiass_index", embedding_model, allow_dangerous_deserialization=True, distance_strategy=DistanceStrategy.COSINE)

# Теперь вы можете сделать запрос на похожие документы
query = "Заключенные в гулагах"
docs = vectorstore.similarity_search(query, k=5)  # k - сколько результатов получить

for doc in docs:
    print("Содержимое документа:", doc.page_content)
    print("Метаданные:", doc.metadata)
    print("------")

Содержимое документа: Федеральная служба исполнения наказаний
* «Росомаха» — отдел специального назначения УФСИН России по Ямало-Ненецкому автономному округу;
Статистика
Тенденция снижения численности заключенных, 2000—2023
|Год | |Общая численность заключённых | |Доля заключённых на 100 тыс. населения
|Год | |Общая численность заключённых | |Доля заключённых на 100 тыс. населения
2000 | 1 060 404 | 729
2002 | 980 151 | 675
2004 | 847 004 | 588
2006 | 823 403 | 577
2008 | 883 436 | 622
2010 | 864 197 | 609
2012 | 755 651 | 528
2014 | 677 287 | 471
2016 | 646 085 | 448
2018 | 602 176 | 416
2020 | 523 928 | 363
2021 | 472 226 | 327
2022 | 468 237 | 324
2023 | 433 000 | 300
Метаданные: {'uuid': 'wiki-1035831-Федеральная_служба_исполнения_наказаний-315858', 'header': ['|Год', '|Общая численность заключённых', '|Доля заключённых на 100 тыс. населения']}
------
Содержимое документа: Азербайджан в Великой Отечественной войне
После войны
Военнопленные
В 1945 году в Азербайджанской ССР

In [64]:
query = "Государственный долг США"
docs = vectorstore.similarity_search(query, k=5)  # k - сколько результатов получить

for doc in docs:
    print("Содержимое документа:", doc.page_content)
    print("Метаданные:", doc.metadata)
    print("------")

Содержимое документа: Государственный долг США
15 ноября 2024 года сумма государственного долга США впервые в истории превысила $36 трлн.
История
Историческая динамика долга
Госдолг США по годам на 31 декабря
Год | Госдолг США, млрд $ | ВВП, млрд $ | Доля госдолга от ВВП, %
Год | Госдолг США, млрд $ | ВВП, млрд $ | Доля госдолга от ВВП, %
1910 | 2,7 | 33,2 | 8,1
1920 | 26,0 | 89,0 | 29,2
1930 | 16,2 | 98,4 | 16,5
1940 | 50,7 | 98,2 | 51,6
1950 | 256,9 | 279,0 | 94,0
1960 | 290,5 | 535,1 | 54,3
1970 | 380,9 | 1049,1 | 36,3
1980 | 909,0 | 2796,8 | 32,5
1990 | 3206,3 | 5914,6 | 54,2
2000 | 5628,7 | 10148,2 | 55,5
2010 | 13 528,7 | 14 798,5 | 91,4
2017 | 20 164,0 | 19 362,1 | 104,1
2020 | 26 000,0 | 20 600,0 | 126,2
2022 | 31 000,0 | 23 000,0 | 134,8
2024 | 35 000,0 | 26 940,0 | 129,8
Метаданные: {'uuid': 'wiki-163538-Государственный_долг_США-72796', 'header': ['Год', 'Госдолг США, млрд $', 'ВВП, млрд $', 'Доля госдолга от ВВП, %']}
------
Содержимое документа: Экономика США
В 1791 году до